In [ ]:
!pip install nltk

In [ ]:
import nltk
from nltk.tag import StanfordNERTagger

# Chemin du modèle Stanford NER
#stanford_classifier = "stanford-ner-4.2.0/classifiers/english.all.3class.distsim.crf.ser.gz"
stanford_classifier = "/Users/zhengruixing/Desktop/stanford-ner-2020-11-17/classifiers/english.all.3class.distsim.crf.ser.gz"
#stanford_classifier = "/Users/zhengruixing/Desktop/stanford-ner-2020-11-17/classifiers/french-ner.ser.gz"  # Modèle français
stanford_ner_path ="/Users/zhengruixing/Desktop/stanford-ner-2020-11-17/stanford-ner.jar"

# Créer le tagger NER
ner_tagger = StanfordNERTagger(stanford_classifier, stanford_ner_path, encoding='utf-8')

# Texte d'exemple
text = "Barack Obama was born in Hawaii. He was the 44th President of the United States."

# Tokenisation
words = nltk.word_tokenize(text)

# Effectuer l'étiquetage NER
ner_results = ner_tagger.tag(words)

# Afficher les résultats
print(ner_results)


[('Barack', 'PERSON'), ('Obama', 'PERSON'), ('was', 'O'), ('born', 'O'), ('in', 'O'), ('Hawaii', 'LOCATION'), ('.', 'O'), ('He', 'O'), ('was', 'O'), ('the', 'O'), ('44th', 'O'), ('President', 'O'), ('of', 'O'), ('the', 'O'), ('United', 'LOCATION'), ('States', 'LOCATION'), ('.', 'O')]


Le modèle Stanford NER génère des résultats d'étiquetage des entités nommées (NER) pour le texte. Chaque tuple contient deux éléments : un mot (ou un symbole) et le type d'entité nommée associé à ce mot.

PERSON indique un nom de personne (comme 'Barack' et 'Obama').
LOCATION indique un nom de lieu (comme 'Hawaii' et 'United States').
O indique une catégorie "autre", ce qui signifie que ce mot n'est pas une entité nommée ou qu'il ne correspond à aucune catégorie spécifique.


In [ ]:
import nltk
from nltk.tag import StanfordNERTagger
import re


stanford_classifier = "/Users/zhengruixing/Desktop/stanford-ner-2020-11-17/classifiers/english.all.3class.distsim.crf.ser.gz"
stanford_ner_path = "/Users/zhengruixing/Desktop/stanford-ner-2020-11-17/stanford-ner.jar"


ner_tagger = StanfordNERTagger(stanford_classifier, stanford_ner_path, encoding='utf-8')

with open("/Users/zhengruixing/Desktop/APPOLINAIRE_Caligrammes.txt", "r", encoding="utf-8") as f:
    texte = f.read().strip()

liste_entitesnom = []
liste_candidat = []

words = nltk.word_tokenize(texte)
ner_results = ner_tagger.tag(words)
for word, tag in ner_results:
    if tag != "O":
        liste_entitesnom.append({"mot": word, "type": tag})
    else:
        liste_candidat.append(word)

print(f"Entités   : {len(liste_entitesnom)}")
print(f"Candidats : {len(liste_candidat )}")
print(liste_entitesnom[:20])
print("-"*50)
print(list(liste_candidat )[:20])


Entités   : 329
Candidats : 15393
[{'mot': 'Texte', 'type': 'ORGANIZATION'}, {'mot': 'Vues', 'type': 'ORGANIZATION'}, {'mot': 'Calligrammes', 'type': 'ORGANIZATION'}, {'mot': '', 'type': 'ORGANIZATION'}, {'mot': 'Guillaume', 'type': 'ORGANIZATION'}, {'mot': 'Apollinaire', 'type': 'ORGANIZATION'}, {'mot': 'Guillaume', 'type': 'PERSON'}, {'mot': 'Lausanne', 'type': 'LOCATION'}, {'mot': 'Roger', 'type': 'PERSON'}, {'mot': 'France', 'type': 'LOCATION'}, {'mot': 'Il', 'type': 'PERSON'}, {'mot': 'GUILLAUME', 'type': 'PERSON'}, {'mot': 'APOLLINAIRE', 'type': 'PERSON'}, {'mot': 'Cordes', 'type': 'PERSON'}, {'mot': 'Cordes', 'type': 'PERSON'}, {'mot': 'Cordes', 'type': 'PERSON'}, {'mot': 'Cordes', 'type': 'PERSON'}, {'mot': 'Bigorneaux', 'type': 'LOCATION'}, {'mot': 'Lotte', 'type': 'LOCATION'}, {'mot': 'Puits', 'type': 'PERSON'}]
--------------------------------------------------
['Rappel', 'de', 'votre', 'demande', ':', 'Format', 'de', 'téléchargement', ':', ':', '1', 'à', '262', 'sur', '262'

Étant donné que Stanford NER n'a pas de modèle de traitement officiel pour le français, le modèle utilisé actuellement est celui en anglais. Cependant, selon les résultats d'annotation, le modèle en anglais génère de nombreuses erreurs de marquage.

#### Stanza est un package de traitement du langage naturel développé par l'Université de Stanford, offrant un support multilingue basé sur l'apprentissage profond, y compris pour le français. Nous pouvons utiliser la bibliothèque Stanza pour effectuer la reconnaissance d'entités nommées. https://github.com/These-SCAI2023/CORPUS/blob/master/prog/Use_stanza.py



In [ ]:
import stanza
def load_stanza_model(lang: str = "fr") -> stanza.Pipeline:
    try:
        nlp = stanza.Pipeline(lang=lang, processors='tokenize,ner')
    except:
        stanza.download(lang=lang, logging_level='DEBUG')
        nlp = stanza.Pipeline(lang=lang, processors='tokenize,ner')
    return nlp

In [ ]:
import nltk
import stanza
import re
import json

# Charger le modèle Stanza
def load_stanza_model(lang: str = "fr") -> stanza.Pipeline:
    try:
        nlp = stanza.Pipeline(lang=lang, processors='tokenize,ner')
    except:
        stanza.download(lang=lang, logging_level='DEBUG')
        nlp = stanza.Pipeline(lang=lang, processors='tokenize,ner')
    return nlp

# Lire le texte à partir d'un fichier
def lire_fichier(chemin, is_json=False):
    with open(chemin, encoding='utf-8') as f:
        if is_json:
            return json.load(f)
        else:
            return f.read().strip()

# Obtenir un dictionnaire représentant une entité nommée
def get_ent_dict(ent) -> dict:
    return {"label": ent.type, "text": ent.text, "jalons": [ent.start_char, ent.end_char]}

# Traiter le texte et retourner un dictionnaire des entités
def dico_resultats(text, lang: str = "fr") -> dict:
    nlp = load_stanza_model(lang=lang)
    doc = nlp(text)
    # Éviter les doublons en se basant sur le texte et le type
    entites_unique = []
    seen_entities = set()  # Pour suivre les entités déjà traitées

    for i, ent in enumerate(doc.ents):
        entity_key = (ent.text, ent.type)
        if entity_key not in seen_entities:
            entites_unique.append({f"entite_{i}": get_ent_dict(ent)})
            seen_entities.add(entity_key)

    return entites_unique

# Obtenir le format d’étiquetage BIO
def bio_stanza(text: str, lang: str = "fr") -> list[str]:
    nlp = load_stanza_model(lang=lang)
    doc = nlp(text)
    return [
        [token.text, token.ner] for sentence in doc.sentences for token in sentence.tokens
    ]

if __name__ == "__main__":
    # Chemin du fichier d'entrée
    file_path = "/Users/zhengruixing/Desktop/APPOLINAIRE_Caligrammes.txt"  # Veuillez le modifier selon votre chemin réel
    texte = lire_fichier(file_path)

    # Obtenir les entités nommées
    entites = dico_resultats(texte, lang="fr")
    print(f"Entités : {len(entites)}")
    print(entites[:20])  # Afficher les 20 premières entités






2025-04-09 19:54:10 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-04-09 19:54:11 INFO: Downloaded file to /Users/zhengruixing/stanza_resources/resources.json
2025-04-09 19:54:11 WARNING: Language fr package default expects mwt, which has been added
2025-04-09 19:54:11 INFO: Loading these models for language: fr (French):
| Processor | Package            |
----------------------------------
| tokenize  | combined           |
| mwt       | combined           |
| ner       | wikinergold_charlm |

2025-04-09 19:54:11 INFO: Using device: cpu
2025-04-09 19:54:11 INFO: Loading: tokenize
2025-04-09 19:54:11 INFO: Loading: mwt
2025-04-09 19:54:11 INFO: Loading: ner
2025-04-09 19:54:13 INFO: Done loading processors!


Entités : 584
[{'entite_0': {'label': 'MISC', 'text': 'Calligrammes', 'jalons': [132, 144]}}, {'entite_1': {'label': 'PER', 'text': 'Guillaume Apollinaire', 'jalons': [147, 168]}}, {'entite_2': {'label': 'PER', 'text': 'La Fresnaye', 'jalons': [182, 193]}}, {'entite_3': {'label': 'PER', 'text': 'Apollinaire', 'jalons': [204, 215]}}, {'entite_4': {'label': 'PER', 'text': 'Guillaume', 'jalons': [217, 226]}}, {'entite_5': {'label': 'LOC', 'text': 'Lausanne', 'jalons': [268, 276]}}, {'entite_7': {'label': 'PER', 'text': 'Roger de', 'jalons': [330, 338]}}, {'entite_8': {'label': 'MISC', 'text': 'Langue : Français', 'jalons': [550, 567]}}, {'entite_9': {'label': 'MISC', 'text': 'Format', 'jalons': [569, 575]}}, {'entite_11': {'label': 'MISC', 'text': 'Description : Collection : Collection du Bouquet', 'jalons': [699, 747]}}, {'entite_13': {'label': 'MISC', 'text': 'Droits : Public domain', 'jalons': [918, 940]}}, {'entite_14': {'label': 'MISC', 'text': 'Identifiant', 'jalons': [942, 953]}}, 

In [ ]:
import json
import os

output_json_path = "/Users/zhengruixing/Desktop/APPOLINAIRE_Caligrammes_entites.json"

# Sauvegarder les entités dans un fichier JSON
with open(output_json_path, "w", encoding="utf-8") as json_file:
    json.dump(entites, json_file, ensure_ascii=False, indent=4)

print(f"Les entités nommées ont été sauvegardées dans : {output_json_path}")


In [ ]:
import nltk
import stanza
import re
import json

# Charger le modèle Stanza
def load_stanza_model(lang: str = "fr") -> stanza.Pipeline:
    try:
        nlp = stanza.Pipeline(lang=lang, processors='tokenize,ner')
    except:
        stanza.download(lang=lang, logging_level='DEBUG')
        nlp = stanza.Pipeline(lang=lang, processors='tokenize,ner')
    return nlp

# Lire le texte à partir d'un fichier
def lire_fichier(chemin, is_json=False):
    with open(chemin, encoding='utf-8') as f:
        if is_json:
            return json.load(f)
        else:
            return f.read().strip()

# Obtenir un dictionnaire représentant une entité nommée (modifié selon la structure demandée)
def get_ent_dict(ent) -> dict:
    return {"mot": ent.text, "type": ent.type}

# Traiter le texte et retourner une liste d'entités sans doublons
def dico_resultats(text, lang: str = "fr") -> dict:
    nlp = load_stanza_model(lang=lang)
    doc = nlp(text)

    entites_unique = []
    seen_entities = set()

    for i, ent in enumerate(doc.ents):
        entity_key = (ent.text, ent.type)
        if entity_key not in seen_entities:
            entites_unique.append(get_ent_dict(ent))  # Stocke uniquement {"mot":..., "type":...}
            seen_entities.add(entity_key)

    return entites_unique

# Obtenir les étiquettes au format BIO
def bio_stanza(text: str, lang: str = "fr") -> list[str]:
    nlp = load_stanza_model(lang=lang)
    doc = nlp(text)
    return [
        [token.text, token.ner] for sentence in doc.sentences for token in sentence.tokens
    ]

def remove_punctuation(token):
    return not re.match(r'[\W_]+', token)

if __name__ == "__main__":
    # Chemin du fichier d'entrée
    file_path = "/Users/zhengruixing/Desktop/APPOLINAIRE_Caligrammes.txt"
    texte = lire_fichier(file_path)

    # Extraire les entités nommées
    entites = dico_resultats(texte, lang="fr")
    print(f"Entités : {len(entites)}")
    print(entites[:20])  # Affiche les 20 premières entités

    # Afficher les tokens obtenus par Stanza
    nlp = load_stanza_model(lang="fr")
    doc = nlp(texte)
    print("\nTokens ：")
    tokens = [token.text for sentence in doc.sentences for token in sentence.tokens]
    print(len(tokens))
    for token in tokens[:20]:
        print(token)

    # Afficher les tokens non reconnus comme entité : candidats
    entite_mots = set(ent["mot"] for ent in entites)
    candidats = []

    for token in tokens:
        if token not in entite_mots and remove_punctuation(token):
            candidats.append(token)

    print("\nCandidats ：")
    print(len(candidats))
    print(candidats[:20])  # Afficher les 20 premiers candidats


2025-04-11 07:11:01 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-04-11 07:11:01 INFO: Downloaded file to /Users/zhengruixing/stanza_resources/resources.json
2025-04-11 07:11:01 WARNING: Language fr package default expects mwt, which has been added
2025-04-11 07:11:01 INFO: Loading these models for language: fr (French):
| Processor | Package            |
----------------------------------
| tokenize  | combined           |
| mwt       | combined           |
| ner       | wikinergold_charlm |

2025-04-11 07:11:01 INFO: Using device: cpu
2025-04-11 07:11:01 INFO: Loading: tokenize
2025-04-11 07:11:01 INFO: Loading: mwt
2025-04-11 07:11:01 INFO: Loading: ner
2025-04-11 07:11:04 INFO: Done loading processors!
2025-04-11 07:11:19 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


Entités : 584
[{'mot': 'Calligrammes', 'type': 'MISC'}, {'mot': 'Guillaume Apollinaire', 'type': 'PER'}, {'mot': 'La Fresnaye', 'type': 'PER'}, {'mot': 'Apollinaire', 'type': 'PER'}, {'mot': 'Guillaume', 'type': 'PER'}, {'mot': 'Lausanne', 'type': 'LOC'}, {'mot': 'Roger de', 'type': 'PER'}, {'mot': 'Langue : Français', 'type': 'MISC'}, {'mot': 'Format', 'type': 'MISC'}, {'mot': 'Description : Collection : Collection du Bouquet', 'type': 'MISC'}, {'mot': 'Droits : Public domain', 'type': 'MISC'}, {'mot': 'Identifiant', 'type': 'MISC'}, {'mot': 'Bibliothèque nationale de France', 'type': 'LOC'}, {'mot': 'Littérature et art', 'type': 'ORG'}, {'mot': 'Conservation numérique', 'type': 'MISC'}, {'mot': 'OCR', 'type': 'MISC'}, {'mot': 'GUILLAUME APOLLINAIRE', 'type': 'PER'}, {'mot': 'CALLIGRAMMES', 'type': 'PER'}, {'mot': 'M E R M O D', 'type': 'LOC'}, {'mot': 'RENÉ DALIZE', 'type': 'PER'}]


2025-04-11 07:11:19 INFO: Downloaded file to /Users/zhengruixing/stanza_resources/resources.json
2025-04-11 07:11:19 WARNING: Language fr package default expects mwt, which has been added
2025-04-11 07:11:19 INFO: Loading these models for language: fr (French):
| Processor | Package            |
----------------------------------
| tokenize  | combined           |
| mwt       | combined           |
| ner       | wikinergold_charlm |

2025-04-11 07:11:19 INFO: Using device: cpu
2025-04-11 07:11:19 INFO: Loading: tokenize
2025-04-11 07:11:19 INFO: Loading: mwt
2025-04-11 07:11:19 INFO: Loading: ner
2025-04-11 07:11:22 INFO: Done loading processors!



Tokens ：
16683
Rappel
de
votre
demande
:
Format
de
téléchargement
:
:
Texte
Vues
1
à
262
sur
262
Nombre
de
pages

Candidats ：
16004
['Rappel', 'de', 'votre', 'demande', 'de', 'téléchargement', 'Texte', 'Vues', '1', 'à', '262', 'sur', '262', 'Nombre', 'de', 'pages', '262', 'Notice', 'complète', 'Titre']


In [ ]:
import nltk
import stanza
import re




# Charger le modèle Stanza une fois pour复用
def load_stanza_model(lang: str = "fr") -> stanza.Pipeline:
    nlp = stanza.Pipeline(lang=lang, processors='tokenize,ner')
    return nlp

# Lire le texte à partir d'un fichier
def lire_fichier(chemin, is_json=False):
    with open(chemin, encoding='utf-8') as f:
        if is_json:
            return json.load(f)
        else:
            return f.read().strip()

# Obtenir un dictionnaire représentant une entité nommée (modifié selon la structure demandée)
def get_ent_dict(ent) -> dict:
    return {"mot": ent.text, "type": ent.type}

# Traiter le texte et retourner une liste d'entités sans doublons
def dico_resultats(text, nlp) -> dict:
    doc = nlp(text)

    entites_unique = []
    seen_entities = set()

    for i, ent in enumerate(doc.ents):
        entity_key = (ent.text, ent.type)
        if entity_key not in seen_entities:
            entites_unique.append(get_ent_dict(ent))  # Stocke uniquement {"mot":..., "type":...}
            seen_entities.add(entity_key)

    return entites_unique

# 去掉标点符号的函数
def remove_punctuation(token):
    return not re.match(r'[\W_]+', token)  # 判断是否是标点符号，若是标点符号则返回 False

# 处理整个corpus的函数
def process_corpus(file_paths, nlp):
    all_entites = []
    all_candidats = []
    all_tokens = []

    for file_path in file_paths:
        texte = lire_fichier(file_path)

        # 获取命名实体
        entites = dico_resultats(texte, nlp)
        all_entites.extend(entites)  # 累积所有命名实体

        # 获取 tokens
        doc = nlp(texte)
        tokens = [token.text for sentence in doc.sentences for token in sentence.tokens]
        all_tokens.extend(tokens)  # 累积所有 tokens

        # 获取未被识别为命名实体的词（candidats）
        entite_mots = set(ent["mot"] for ent in entites)
        for token in tokens:
            if token not in entite_mots and remove_punctuation(token):  # 过滤标点符号
                all_candidats.append(token)  # 累积未识别的词

    return all_entites, all_candidats, all_tokens

if __name__ == "__main__":
    # 文件路径列表
    file_paths = [
        "/Users/zhengruixing/Desktop/APPOLINAIRE_Caligrammes.txt",
        "/Users/zhengruixing/Desktop/DARBOUVILLE_Poesies-et-nouvelles.txt",
        "/Users/zhengruixing/Desktop/DESBORDES-VALMORE_Poesies-1820.txt",
        "/Users/zhengruixing/Desktop/HUGO_Contemplations-T2.txt",
        "/Users/zhengruixing/Desktop/LOISEAU_Fleurs-d-avril.txt",
        "/Users/zhengruixing/Desktop/NOAILLES_Derniers-vers.txt",
        "/Users/zhengruixing/Desktop/RIMBAUD_Illuminations-et-Une-saison-en-enfer-et-Notice-Paul-Verlaine.txt",
        "/Users/zhengruixing/Desktop/VERLAINE_Sagesse.txt",
        "/Users/zhengruixing/Desktop/SAUVAGE_Tandis-que-la-terre-tourne.txt",
        "/Users/zhengruixing/Desktop/VIVIEN_Etudes-et-preludes.txt"
    ]

    # 先加载Stanza模型
    nlp = load_stanza_model(lang="fr")

    # 处理整个语料库
    all_entites, all_candidats, all_tokens = process_corpus(file_paths, nlp)

    # 输出结果
    print(f"Entités : {len(all_entites)}")
    print(all_entites[:20])  # 打印前20个命名实体

    print(f"\nCandidats : {len(all_candidats)}")
    print(all_candidats[:20])  # 打印前20个未识别的词

    print(f"\nTokens : {len(all_tokens)}")
    print(all_tokens[:20])  # 打印前20个tokens


2025-04-11 07:25:54 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-04-11 07:25:55 INFO: Downloaded file to /Users/zhengruixing/stanza_resources/resources.json
2025-04-11 07:25:55 WARNING: Language fr package default expects mwt, which has been added
2025-04-11 07:25:55 INFO: Loading these models for language: fr (French):
| Processor | Package            |
----------------------------------
| tokenize  | combined           |
| mwt       | combined           |
| ner       | wikinergold_charlm |

2025-04-11 07:25:55 INFO: Using device: cpu
2025-04-11 07:25:55 INFO: Loading: tokenize
2025-04-11 07:25:55 INFO: Loading: mwt
2025-04-11 07:25:55 INFO: Loading: ner
2025-04-11 07:25:57 INFO: Done loading processors!


KeyboardInterrupt: 

In [ ]:
import nltk
import stanza
import re
import glob

# Charger le modèle Stanza une fois pour复用
def load_stanza_model(lang: str = "fr") -> stanza.Pipeline:
    nlp = stanza.Pipeline(lang=lang, processors='tokenize,ner')
    return nlp

# Lire le texte à partir d'un fichier
def lire_fichier(chemin, is_json=False):
    with open(chemin, encoding='utf-8') as f:
        if is_json:
            return json.load(f)
        else:
            return f.read().strip()

# Obtenir un dictionnaire représentant une entité nommée (modifié selon la structure demandée)
def get_ent_dict(ent) -> dict:
    return {"mot": ent.text, "type": ent.type}

# Traiter le texte et retourner une liste d'entités sans doublons
def dico_resultats(text, nlp) -> dict:
    doc = nlp(text)

    entites_unique = []
    seen_entities = set()

    for i, ent in enumerate(doc.ents):
        entity_key = (ent.text, ent.type)
        if entity_key not in seen_entities:
            entites_unique.append(get_ent_dict(ent))  # Stocke uniquement {"mot":..., "type":...}
            seen_entities.add(entity_key)

    return entites_unique

# 去掉标点符号的函数
def remove_punctuation(token):
    return not re.match(r'[\W_]+', token)  # 判断是否是标点符号，若是标点符号则返回 False

# 处理整个corpus的函数
def process_corpus(file_paths, nlp):
    all_entites = []
    all_candidats = []
    all_tokens = []

    for file_path in file_paths:
        texte = lire_fichier(file_path)

        # 获取命名实体
        entites = dico_resultats(texte, nlp)
        all_entites.extend(entites)  # 累积所有命名实体

        # 获取 tokens
        doc = nlp(texte)
        tokens = [token.text for sentence in doc.sentences for token in sentence.tokens]
        all_tokens.extend(tokens)  # 累积所有 tokens

        # 获取未被识别为命名实体的词（candidats）
        entite_mots = set(ent["mot"] for ent in entites)
        for token in tokens:
            if token not in entite_mots and remove_punctuation(token):  # 过滤标点符号
                all_candidats.append(token)  # 累积未识别的词

    return all_entites, all_candidats, all_tokens

if __name__ == "__main__":
    # 手动列出的文件路径
    manual_file_paths = [
        "/Users/zhengruixing/Desktop/APPOLINAIRE_Caligrammes.txt",
        "/Users/zhengruixing/Desktop/DARBOUVILLE_Poesies-et-nouvelles.txt",
        "/Users/zhengruixing/Desktop/DESBORDES-VALMORE_Poesies-1820.txt",
        "/Users/zhengruixing/Desktop/HUGO_Contemplations-T2.txt",
        "/Users/zhengruixing/Desktop/LOISEAU_Fleurs-d-avril.txt",
        "/Users/zhengruixing/Desktop/NOAILLES_Derniers-vers.txt",
        "/Users/zhengruixing/Desktop/RIMBAUD_Illuminations-et-Une-saison-en-enfer-et-Notice-Paul-Verlaine.txt",
        "/Users/zhengruixing/Desktop/VERLAINE_Sagesse.txt",
        "/Users/zhengruixing/Desktop/SAUVAGE_Tandis-que-la-terre-tourne.txt",
        "/Users/zhengruixing/Desktop/VIVIEN_Etudes-et-preludes.txt"
    ]

    # 使用glob获取目录下的所有txt文件
    additional_file_paths = glob.glob("/Users/zhengruixing/Desktop/Corpus/*.txt")

    # 合并手动列出的路径和通过glob获取的路径
    file_paths = manual_file_paths + additional_file_paths

    # 输出文件路径确认
    print("File paths to process:")
    for path in file_paths:
        print(path)

    # 先加载Stanza模型
    nlp = load_stanza_model(lang="fr")

    # 处理整个语料库
    all_entites, all_candidats, all_tokens = process_corpus(file_paths, nlp)

    # 输出结果
    print(f"Entités : {len(all_entites)}")
    print(all_entites[:20])  # 打印前20个命名实体

    print(f"\nCandidats : {len(all_candidats)}")
    print(all_candidats[:20])  # 打印前20个未识别的词

    print(f"\nTokens : {len(all_tokens)}")
    print(all_tokens[:20])  # 打印前20个tokens



2025-04-11 07:51:43 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


File paths to process:
/Users/zhengruixing/Desktop/APPOLINAIRE_Caligrammes.txt
/Users/zhengruixing/Desktop/DARBOUVILLE_Poesies-et-nouvelles.txt
/Users/zhengruixing/Desktop/DESBORDES-VALMORE_Poesies-1820.txt
/Users/zhengruixing/Desktop/HUGO_Contemplations-T2.txt
/Users/zhengruixing/Desktop/LOISEAU_Fleurs-d-avril.txt
/Users/zhengruixing/Desktop/NOAILLES_Derniers-vers.txt
/Users/zhengruixing/Desktop/RIMBAUD_Illuminations-et-Une-saison-en-enfer-et-Notice-Paul-Verlaine.txt
/Users/zhengruixing/Desktop/VERLAINE_Sagesse.txt
/Users/zhengruixing/Desktop/SAUVAGE_Tandis-que-la-terre-tourne.txt
/Users/zhengruixing/Desktop/VIVIEN_Etudes-et-preludes.txt


2025-04-11 07:51:43 INFO: Downloaded file to /Users/zhengruixing/stanza_resources/resources.json
2025-04-11 07:51:43 WARNING: Language fr package default expects mwt, which has been added
2025-04-11 07:51:44 INFO: Loading these models for language: fr (French):
| Processor | Package            |
----------------------------------
| tokenize  | combined           |
| mwt       | combined           |
| ner       | wikinergold_charlm |

2025-04-11 07:51:44 INFO: Using device: cpu
2025-04-11 07:51:44 INFO: Loading: tokenize
2025-04-11 07:51:44 INFO: Loading: mwt
2025-04-11 07:51:44 INFO: Loading: ner
2025-04-11 07:51:46 INFO: Done loading processors!


Entités : 8116
[{'mot': 'Calligrammes', 'type': 'MISC'}, {'mot': 'Guillaume Apollinaire', 'type': 'PER'}, {'mot': 'La Fresnaye', 'type': 'PER'}, {'mot': 'Apollinaire', 'type': 'PER'}, {'mot': 'Guillaume', 'type': 'PER'}, {'mot': 'Lausanne', 'type': 'LOC'}, {'mot': 'Roger de', 'type': 'PER'}, {'mot': 'Langue : Français', 'type': 'MISC'}, {'mot': 'Format', 'type': 'MISC'}, {'mot': 'Description : Collection : Collection du Bouquet', 'type': 'MISC'}, {'mot': 'Droits : Public domain', 'type': 'MISC'}, {'mot': 'Identifiant', 'type': 'MISC'}, {'mot': 'Bibliothèque nationale de France', 'type': 'LOC'}, {'mot': 'Littérature et art', 'type': 'ORG'}, {'mot': 'Conservation numérique', 'type': 'MISC'}, {'mot': 'OCR', 'type': 'MISC'}, {'mot': 'GUILLAUME APOLLINAIRE', 'type': 'PER'}, {'mot': 'CALLIGRAMMES', 'type': 'PER'}, {'mot': 'M E R M O D', 'type': 'LOC'}, {'mot': 'RENÉ DALIZE', 'type': 'PER'}]

Candidats : 232577
['Rappel', 'de', 'votre', 'demande', 'de', 'téléchargement', 'Texte', 'Vues', '1',

In [ ]:
 result = {
        "entites": all_entites,
        "candidats": all_candidats,
        "tokens": all_tokens
    }


with open("/Users/zhengruixing/Desktop/0411/tous_entites_nommes_Stanza.json", "w", encoding="utf-8") as json_file:
        json.dump(result, json_file, indent=4, ensure_ascii=False)

In [ ]:
import nltk
import stanza
import re
import json
import os
from tqdm import tqdm  # Importer la bibliothèque tqdm
import time  # Pour calculer le temps de traitement de chaque fichier

# Charger le modèle Stanza une fois pour réutilisation
def load_stanza_model(lang: str = "fr") -> stanza.Pipeline:
    nlp = stanza.Pipeline(lang=lang, processors='tokenize,ner')
    return nlp

# Lire le texte à partir d'un fichier
def lire_fichier(chemin, is_json=False):
    with open(chemin, encoding='utf-8') as f:
        if is_json:
            return json.load(f)
        else:
            return f.read().strip()

# Obtenir un dictionnaire représentant une entité nommée (modifié selon la structure demandée)
def get_ent_dict(ent) -> dict:
    return {"mot": ent.text, "type": ent.type}

# Traiter le texte et retourner une liste d'entités sans doublons
def dico_resultats(text, nlp) -> dict:
    doc = nlp(text)

    entites_unique = []
    seen_entities = set()

    for i, ent in enumerate(doc.ents):
        entity_key = (ent.text, ent.type)
        if entity_key not in seen_entities:
            entites_unique.append(get_ent_dict(ent))  # Stocke uniquement {"mot":..., "type":...}
            seen_entities.add(entity_key)

    return entites_unique

# Fonction pour supprimer la ponctuation
def remove_punctuation(token):
    return not re.match(r'[\W_]+', token)  # Vérifie si c'est un signe de ponctuation, retourne False si c'est le cas

# Traiter l'ensemble du corpus
def process_corpus(file_paths, output_dir, nlp):
    os.makedirs(output_dir, exist_ok=True)  # Créer le répertoire de sortie si il n'existe pas

    # Utiliser tqdm pour afficher une barre de progression
    for file_path in tqdm(file_paths, desc="Traitement des fichiers", unit="fichier"):
        start_time = time.time()  # Enregistrer l'heure de début du traitement

        # Obtenir le nom du fichier
        file_name = os.path.basename(file_path).split('.')[0]

        # Lire le contenu du fichier
        texte = lire_fichier(file_path)

        # Extraire les entités nommées
        entites = dico_resultats(texte, nlp)

        # Extraire les tokens
        doc = nlp(texte)
        tokens = [token.text for sentence in doc.sentences for token in sentence.tokens]

        # Extraire les mots qui ne sont pas des entités nommées (candidats)
        entite_mots = set(ent["mot"] for ent in entites)
        candidats = [token for token in tokens if token not in entite_mots and remove_punctuation(token)]

        # Générer les données de sortie
        output_data = {
            "entites_nom": entites,
            "candidats": candidats
        }

        # Sauvegarder les résultats dans un fichier JSON
        json_file_path = os.path.join(output_dir, f"{file_name}_entites_candidats.json")
        with open(json_file_path, "w", encoding="utf-8") as json_file:
            json.dump(output_data, json_file, indent=4, ensure_ascii=False)

        # Calculer et afficher le temps de traitement pour chaque fichier
        processing_time = time.time() - start_time
        print(f"Fichier : {file_name}")
        print(f"  Tokens: {len(tokens)}")
        print(f"  Entités: {len(entites)}")
        print(f"  Candidats: {len(candidats)}")
        print(f"  Temps de traitement : {processing_time:.2f} secondes")
        print("-" * 20)

        # Afficher les 20 premières entités et candidats
        print(f"  20 premières entités : {entites[:20]}")
        print(f"  20 premiers candidats : {candidats[:20]}")
        print("=" * 40)

        # Sauvegarder les entités dans un fichier texte
        with open(os.path.join(output_dir, f"{file_name}_entites.txt"), "w", encoding="utf-8") as entites_file:
            for entite in entites:
                entites_file.write(f"{entite['mot']} - {entite['type']}\n")

        # Sauvegarder les candidats dans un fichier texte
        with open(os.path.join(output_dir, f"{file_name}_candidats.txt"), "w", encoding="utf-8") as candidats_file:
            for mot in candidats:
                candidats_file.write(f"{mot}\n")

    # Afficher un message lorsque tous les fichiers sont traités
    print("Traitement terminé pour tous les fichiers.")

if __name__ == "__main__":
    # Liste des chemins des fichiers
    file_paths = [
        "/Users/zhengruixing/Desktop/APPOLINAIRE_Caligrammes.txt",
        "/Users/zhengruixing/Desktop/DARBOUVILLE_Poesies-et-nouvelles.txt",
        "/Users/zhengruixing/Desktop/DESBORDES-VALMORE_Poesies-1820.txt",
        "/Users/zhengruixing/Desktop/HUGO_Contemplations-T2.txt",
        "/Users/zhengruixing/Desktop/LOISEAU_Fleurs-d-avril.txt",
        "/Users/zhengruixing/Desktop/NOAILLES_Derniers-vers.txt",
        "/Users/zhengruixing/Desktop/RIMBAUD_Illuminations-et-Une-saison-en-enfer-et-Notice-Paul-Verlaine.txt",
        "/Users/zhengruixing/Desktop/VERLAINE_Sagesse.txt",
        "/Users/zhengruixing/Desktop/SAUVAGE_Tandis-que-la-terre-tourne.txt",
        "/Users/zhengruixing/Desktop/VIVIEN_Etudes-et-preludes.txt"
    ]

    # Répertoire de sortie
    output_dir = "/Users/zhengruixing/Desktop/0411/Stanza_output"

    # Charger le modèle Stanza
    nlp = load_stanza_model(lang="fr")

    # Traiter l'ensemble du corpus et sauvegarder les résultats
    process_corpus(file_paths, output_dir, nlp)


2025-04-11 07:37:59 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-04-11 07:37:59 INFO: Downloaded file to /Users/zhengruixing/stanza_resources/resources.json
2025-04-11 07:37:59 WARNING: Language fr package default expects mwt, which has been added
2025-04-11 07:38:00 INFO: Loading these models for language: fr (French):
| Processor | Package            |
----------------------------------
| tokenize  | combined           |
| mwt       | combined           |
| ner       | wikinergold_charlm |

2025-04-11 07:38:00 INFO: Using device: cpu
2025-04-11 07:38:00 INFO: Loading: tokenize
2025-04-11 07:38:00 INFO: Loading: mwt
2025-04-11 07:38:00 INFO: Loading: ner
2025-04-11 07:38:02 INFO: Done loading processors!
Processing files:  10%|██▍                     | 1/10 [00:31<04:47, 31.99s/file]

Fichier : APPOLINAIRE_Caligrammes
  Tokens: 16683
  Entités: 584
  Candidats: 16004
  Temps de traitement : 31.99 secondes
--------------------
  20 premières entités : [{'mot': 'Calligrammes', 'type': 'MISC'}, {'mot': 'Guillaume Apollinaire', 'type': 'PER'}, {'mot': 'La Fresnaye', 'type': 'PER'}, {'mot': 'Apollinaire', 'type': 'PER'}, {'mot': 'Guillaume', 'type': 'PER'}, {'mot': 'Lausanne', 'type': 'LOC'}, {'mot': 'Roger de', 'type': 'PER'}, {'mot': 'Langue : Français', 'type': 'MISC'}, {'mot': 'Format', 'type': 'MISC'}, {'mot': 'Description : Collection : Collection du Bouquet', 'type': 'MISC'}, {'mot': 'Droits : Public domain', 'type': 'MISC'}, {'mot': 'Identifiant', 'type': 'MISC'}, {'mot': 'Bibliothèque nationale de France', 'type': 'LOC'}, {'mot': 'Littérature et art', 'type': 'ORG'}, {'mot': 'Conservation numérique', 'type': 'MISC'}, {'mot': 'OCR', 'type': 'MISC'}, {'mot': 'GUILLAUME APOLLINAIRE', 'type': 'PER'}, {'mot': 'CALLIGRAMMES', 'type': 'PER'}, {'mot': 'M E R M O D', 'ty

Processing files:  20%|████▊                   | 2/10 [02:10<09:31, 71.38s/file]

Fichier : DARBOUVILLE_Poesies-et-nouvelles
  Tokens: 55462
  Entités: 1064
  Candidats: 42992
  Temps de traitement : 98.94 secondes
--------------------
  20 premières entités : [{'mot': 'Titre : Poésies et nouvelles', 'type': 'MISC'}, {'mot': "Madame d'Arbouville", 'type': 'PER'}, {'mot': "Sophie d'Arbouville", 'type': 'PER'}, {'mot': 'P. de Barante', 'type': 'PER'}, {'mot': 'Arbouville', 'type': 'PER'}, {'mot': 'Sophie', 'type': 'PER'}, {'mot': 'Amyot', 'type': 'PER'}, {'mot': 'Paris', 'type': 'LOC'}, {'mot': 'Barante', 'type': 'PER'}, {'mot': 'Prosper Brugière', 'type': 'PER'}, {'mot': 'Format', 'type': 'MISC'}, {'mot': 'Droits : Public domain', 'type': 'MISC'}, {'mot': 'Identifiant', 'type': 'MISC'}, {'mot': 'Bibliothèque nationale de France', 'type': 'LOC'}, {'mot': 'Littérature et art', 'type': 'ORG'}, {'mot': 'Conservation numérique', 'type': 'MISC'}, {'mot': 'OCR', 'type': 'MISC'}, {'mot': 'NOUVELLES', 'type': 'ORG'}, {'mot': 'TYPOGRAPHIE DE CH. LAHURE Imprimeur du Sénat', 'ty

...

Fichier : VERLAINE_Sagesse
  Tokens: 14122
  Entités: 603
  Candidats: 11213
  Temps de traitement : 27.22 secondes
--------------------
  20 premières entités : [{'mot': 'Titre : Sagesse', 'type': 'MISC'}, {'mot': 'Paul Verlaine', 'type': 'PER'}, {'mot': 'Verlaine', 'type': 'PER'}, {'mot': 'Paul', 'type': 'PER'}, {'mot': 'L. Vanier', 'type': 'PER'}, {'mot': 'Paris', 'type': 'LOC'}, {'mot': 'Langue : Français', 'type': 'MISC'}, {'mot': 'Format', 'type': 'MISC'}, {'mot': 'Sagesse', 'type': 'PER'}, {'mot': 'Droits : Public domain', 'type': 'MISC'}, {'mot': 'Identifiant', 'type': 'MISC'}, {'mot': 'Bibliothèque nationale de France', 'type': 'LOC'}, {'mot': 'Littérature et art', 'type': 'ORG'}, {'mot': 'Conservation numérique', 'type': 'MISC'}, {'mot': 'OCR', 'type': 'MISC'}, {'mot': 'PAUL VERLAINE', 'type': 'PER'}, {'mot': 'SAGESSE', 'type': 'ORG'}, {'mot': 'REVUE El CORUIUKE', 'type': 'ORG'}, {'mot': 'PARIS', 'type': 'ORG'}, {'mot': 'EH', 'type': 'ORG'}]
  20 premiers candidats : ['Rappel

Processing files:  90%|█████████████████████▌  | 9/10 [08:02<00:42, 42.98s/file]

Fichier : SAUVAGE_Tandis-que-la-terre-tourne
  Tokens: 20726
  Entités: 481
  Candidats: 18190
  Temps de traitement : 41.73 secondes
--------------------
  20 premières entités : [{'mot': 'Titre : Tandis que la terre', 'type': 'MISC'}, {'mot': 'Cécile Sauvage', 'type': 'PER'}, {'mot': 'Sauvage', 'type': 'PER'}, {'mot': 'Cécile', 'type': 'PER'}, {'mot': 'Mercure de France', 'type': 'MISC'}, {'mot': 'Paris', 'type': 'LOC'}, {'mot': 'Format', 'type': 'MISC'}, {'mot': 'Droits : Public domain', 'type': 'MISC'}, {'mot': 'Identifiant', 'type': 'MISC'}, {'mot': 'Bibliothèque nationale de France', 'type': 'LOC'}, {'mot': 'Littérature et art', 'type': 'ORG'}, {'mot': 'Conservation numérique', 'type': 'MISC'}, {'mot': 'OCR', 'type': 'MISC'}, {'mot': 'CÉCILE SAUVAGE', 'type': 'PER'}, {'mot': 'POEMES', 'type': 'MISC'}, {'mot': 'PARIS MERCVRE DE FRANGE XXVI', 'type': 'ORG'}, {'mot': 'RVE DE CONDÉ', 'type': 'ORG'}, {'mot': 'XXVI', 'type': 'ORG'}, {'mot': 'Hollande', 'type': 'PER'}, {'mot': 'TIRAGE',

Processing files: 100%|███████████████████████| 10/10 [09:17<00:00, 55.75s/file]

Fichier : VIVIEN_Etudes-et-preludes
  Tokens: 38881
  Entités: 1468
  Candidats: 29875
  Temps de traitement : 75.34 secondes
--------------------
  20 premières entités : [{'mot': 'Titre : Poèmes de Renée Vivien : Études et préludes', 'type': 'MISC'}, {'mot': 'Cendres et poussières', 'type': 'MISC'}, {'mot': 'Évocations', 'type': 'MISC'}, {'mot': 'Sapho', 'type': 'PER'}, {'mot': 'La Vénus des aveugles', 'type': 'MISC'}, {'mot': 'Vivien', 'type': 'PER'}, {'mot': 'Renée', 'type': 'PER'}, {'mot': 'A. Lemerre', 'type': 'PER'}, {'mot': 'Paris', 'type': 'LOC'}, {'mot': 'Langue : Français', 'type': 'MISC'}, {'mot': 'Format', 'type': 'MISC'}, {'mot': 'XML DTBook', 'type': 'MISC'}, {'mot': 'Droits : Public domain', 'type': 'MISC'}, {'mot': 'Identifiant', 'type': 'MISC'}, {'mot': 'Bibliothèque nationale de France', 'type': 'LOC'}, {'mot': 'Littérature et art', 'type': 'ORG'}, {'mot': 'Conservation numérique', 'type': 'MISC'}, {'mot': 'OCR', 'type': 'MISC'}, {'mot': 'POÈMES', 'type': 'MISC'}, {'

### Exclure les limitations de Stanza

Stanza utilise effectivement des modèles basés sur des architectures similaires à BERT (comme celles du cadre transformers), et ces modèles peuvent être soumis à une limitation de la longueur d'entrée, ce qui signifie que les textes plus longs risquent de dépasser cette limite. En conséquence, Stanza pourrait être amené à diviser les textes en segments plus petits afin de respecter cette limite de longueur d'entrée.

Cependant, Stanza ne fournit pas de documentation précise sur la longueur maximale de chaque entrée pour chaque modèle qu'il utilise. En règle générale, cela dépend de l'implémentation du modèle sous-jacent. Pour la plupart des modèles, cette gestion est automatisée, ce qui signifie que Stanza s'occupe de diviser le texte en segments appropriés de manière interne, sans que l'utilisateur ait à se soucier de cette question. Cela garantit que le texte ne dépasse pas la longueur maximale autorisée par le modèle sous-jacent.

Stanza 在其处理过程中确实会利用类似于 BERT 的模型（如 transformers 库的实现），因此也可能面临 最大输入长度 的限制。这意味着，在处理较长的文本时，Stanza 也可能将文本切分成较小的片段进行处理，以适应最大输入长度的要求。

但是，Stanza 并没有明确的文档说明对每个输入句子的长度限制是多少，通常这依赖于底层模型的具体实现。大多数情况下，Stanza 会在其底层模型的基础上自动处理这一点，所以开发者不需要显式地拆分句子。其文本处理过程会进行适当的拆分，以避免超出模型的输入长度限制。